In [185]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [186]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [187]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[2]

ticker_upper = ticker.upper()

print(ticker_upper)

MSFT


In [188]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [189]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek


df["return"] = df["5. adjusted close"].pct_change(1)
df["increase"] = df["return"].apply(lambda x: 1 if x > 0 else 0)

df["log_return"] = np.log(df["return"] + 1)


df.set_index("date", inplace=True)

In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  return                3063 non-null   float64       
 11  increase              3064 non-null   int64         
 12  log_return            3063 non-null   float64       
dtypes: datet

In [191]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,NaN,0,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,0.000323,1,0.000323
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.006137,0,-0.006156
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,-0.010335,0,-0.010389
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,0.006830,1,0.006807


In [192]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,0.004978,1,0.004966
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,-0.012852,0,-0.012935
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,0.017766,1,0.017610
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.014224,0,-0.014326
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0,-0.020691


In [193]:
df["return"] = df["return"].shift(-1)
df["increase"] = df["increase"].shift(-1)

In [194]:
df.head()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807


In [195]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,NaN,NaN,-0.020691


In [196]:
df.increase.value_counts()

1.0    1608
0.0    1455
Name: increase, dtype: int64

In [197]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [198]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [199]:
benzinga_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_20100101_20220304.csv"

In [200]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [201]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['Stock'])

In [202]:
df_benzinga["day_of_week"] = df_benzinga["date"].dt.dayofweek

In [203]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     11106 non-null  object        
 1   Stock        11106 non-null  object        
 2   Positive     11106 non-null  float64       
 3   Negative     11106 non-null  float64       
 4   Neutral      11106 non-null  float64       
 5   date         11106 non-null  datetime64[ns]
 6   day_of_week  11106 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 607.5+ KB


In [204]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Stock Market News for January 04, 2010 - Market News U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50. During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express , a 56.8% surge in Microsoft , and a 55.5% gain in IBM . Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday. Of note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.Zacks Investment Research",2010-01-04,0.208157,0.760072,0.031771,2010-01-04,0


In [205]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     11106 non-null  object        
 1   Stock        11106 non-null  object        
 2   Positive     11106 non-null  float64       
 3   Negative     11106 non-null  float64       
 4   Neutral      11106 non-null  float64       
 5   date         11106 non-null  datetime64[ns]
 6   day_of_week  11106 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 607.5+ KB


In [206]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_benzinga.day_of_week.value_counts()

2    2278
3    2237
1    2222
0    1984
4    1887
6    276 
5    222 
Name: day_of_week, dtype: int64

In [207]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Stock Market News for January 04, 2010 - Market News U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50. During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express , a 56.8% surge in Microsoft , and a 55.5% gain in IBM . Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday. Of note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.Zacks Investment Research",2010-01-04,0.208157,0.760072,0.031771,2010-01-04,0


In [208]:
df_benzinga["in_index"] = df_benzinga["Stock"].apply(lambda x: True if x in df.index else False)

In [209]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    276
5    222
0    39 
4    7  
1    3  
3    2  
2    1  
Name: day_of_week, dtype: int64

In [210]:
df.index.get_loc('2010-01-04')


0

In [211]:
from datetime import date, datetime, timedelta

In [212]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [213]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Monday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Monday
                days = timedelta(2)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Monday
                days = timedelta(3)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Tuesday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(e3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc

In [214]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_benzinga["backfill_date"] = df_benzinga.apply(update_date, axis=1)


2010-01-11
2010-01-18
'2010-01-18'
2010-01-19
2010-01-19
2010-01-19
2010-01-25
2010-02-08
2010-02-15
'2010-02-15'
2010-02-16
2010-02-16
2010-03-08
2010-03-08
2010-03-15
2010-03-29
2010-03-29
2010-04-05
2010-04-05
2010-05-24
2010-05-31
'2010-05-31'
2010-07-12
2010-07-19
2010-08-09
2010-09-06
'2010-09-06'
2010-09-07
2010-10-11
2010-11-08
2010-11-29
2010-12-13
2010-12-13
2011-01-10
2011-01-18
2011-01-18
2011-01-24
2011-01-24
2011-03-28
2011-04-18
2011-04-18
2011-04-18
2011-04-25
2011-04-25
2011-05-02
2011-05-16
2011-05-16
2011-05-23
2011-06-06
2011-06-20
2011-07-18
2011-07-18
2011-06-27
2011-08-22
2011-09-19
2011-10-03
2011-10-17
2011-10-24
2012-01-09
2012-01-16
'2012-01-16'
2012-02-20
'2012-02-20'
2012-02-21
2012-02-27
2012-03-05
2012-03-05
2012-04-09
2012-04-09
2012-04-16
2012-04-16
2012-04-23
2012-05-28
'2012-05-28'
2012-05-29
2012-06-11
2012-06-18
2012-06-18
2012-06-25
2012-07-16
2012-09-10
2012-09-17
2012-10-01
2012-10-08
2012-10-22
2012-10-29
'2012-10-29'
2012-10-30
'2012-10-30'
201

In [215]:
ERROR_COUNTER

45

In [216]:
df_benzinga["in_index"] = df_benzinga["backfill_date"].apply(lambda x: True if x in df.index else False)

In [217]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    18
5    16
0    11
1    3 
3    2 
2    1 
Name: day_of_week, dtype: int64

In [218]:
df_benzinga[df_benzinga.in_index==False].shape

(51, 9)

In [219]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [220]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [221]:
df_benzinga = df_benzinga[["backfill_date", "Positive", "Negative", "Neutral"]]

In [222]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  11106 non-null  object 
 1   Positive       11106 non-null  float64
 2   Negative       11106 non-null  float64
 3   Neutral        11106 non-null  float64
dtypes: float64(3), object(1)
memory usage: 347.2+ KB


In [223]:
df_benzinga.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-04,0.208157,0.760072,0.031771
1,2010-01-04,0.024145,0.808129,0.167726
2,2010-01-04,0.922880,0.023758,0.053362
3,2010-01-04,0.143269,0.012789,0.843941
4,2010-01-05,0.874124,0.063757,0.062120


In [224]:
df_benzinga.rename(columns={"backfill_date": "date"}, inplace=True)

In [225]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [226]:
# df_benzinga = clean_tweet(df_benzinga)

In [227]:
df_benzinga.set_index("date", inplace=True)

In [228]:
df_benzinga.head()

,Positive,Negative,Neutral
date,,,
2010-01-04,0.208157,0.760072,0.031771
2010-01-04,0.024145,0.808129,0.167726
2010-01-04,0.922880,0.023758,0.053362
2010-01-04,0.143269,0.012789,0.843941
2010-01-05,0.874124,0.063757,0.062120


In [229]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [230]:
df_benzinga_duplicated_index.shape[0]

10564

In [231]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [232]:
df_benzinga_nonduplicated_index.shape[0]

542

In [233]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [234]:
df_benzinga_groupby

,negative,nuetral,positive
date,,,
2010-01-04,0.401187,0.274200,0.324613
2010-01-05,0.108230,0.176608,0.715163
2010-01-06,0.050018,0.339652,0.610330
2010-01-07,0.029928,0.195104,0.774968
2010-01-08,0.319698,0.402485,0.277817
...,...,...,...
2022-02-28,0.284602,0.617978,0.097420
2022-03-01,0.083546,0.596145,0.320308
2022-03-02,0.522847,0.434464,0.042689


In [235]:
df_merge = pd.merge(df, df_benzinga_groupby, how="left", left_index=True, right_index=True)

In [236]:
df_merge.shape

(3064, 16)

In [237]:
df_merge.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [238]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-22,30.000,30.2000,28.8400,28.96,22.323385,102004600.0,0.00,1.0,2010-01-22,4,0.012431,1.0,-0.035615,NaN,NaN,NaN
2010-02-09,27.970,28.3400,27.7500,28.01,21.591091,59195800.0,0.00,1.0,2010-02-09,1,-0.000714,0.0,0.010407,NaN,NaN,NaN
2010-02-11,27.930,28.4000,27.7000,28.12,21.675883,65993700.0,0.00,1.0,2010-02-11,3,-0.006757,0.0,0.004634,NaN,NaN,NaN
2010-02-22,28.840,28.9400,28.6500,28.73,22.247645,36707100.0,0.00,1.0,2010-02-22,0,-0.013923,0.0,-0.001391,NaN,NaN,NaN
2010-02-26,28.650,28.8500,28.5100,28.67,22.201182,40370600.0,0.00,1.0,2010-02-26,4,0.012208,1.0,0.002445,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-02,302.200,303.3602,300.1800,301.15,299.979431,16290545.0,0.00,1.0,2021-09-02,3,-0.000033,0.0,-0.002255,NaN,NaN,NaN
2021-10-15,302.340,304.4500,300.5191,304.21,303.027537,25384803.0,0.00,1.0,2021-10-15,4,0.010125,1.0,0.004811,NaN,NaN,NaN
2021-11-30,335.320,337.7800,328.9900,330.59,329.907054,42885609.0,0.00,1.0,2021-11-30,1,-0.001543,0.0,-0.018105,NaN,NaN,NaN


In [239]:
df.shape

(3064, 13)

In [240]:
df_benzinga_groupby.shape

(2746, 3)

In [241]:
# df_merge.fillna(0.0, inplace=True)
df_merge.fillna(method="ffill", limit=1, inplace=True)

In [242]:
df_merge[df_merge.negative.isnull()].shape

(59, 16)

In [243]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  return                3064 non-null   float64       
 11  increase              3064 non-null   float64       
 12  log_return            3063 non-null   float64       
 13  negative

In [244]:
df_merge.fillna(0.0, inplace=True)

In [245]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,0.000000,0.401187,0.274200,0.324613
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,0.108230,0.176608,0.715163
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,0.050018,0.339652,0.610330
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,0.029928,0.195104,0.774968
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807,0.319698,0.402485,0.277817


In [246]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [247]:
df_merge.shape

(3064, 16)

In [248]:
# df_merge[(df_merge["negative"]==0.0) & (df_merge["nuetral"]==0.0) & (df_merge["positive"]==0.0)].shape

In [249]:
# df_merge = df_merge[~(df_merge["negative"]==0.0) & ~(df_merge["nuetral"]==0.0) & ~(df_merge["positive"]==0.0)].copy()

In [250]:
X = df_merge[["log_return", "6. volume", "positive"]]
y = df_merge["increase"]

In [251]:
y.head()

date
2010-01-04    1.0
2010-01-05    0.0
2010-01-06    0.0
2010-01-07    1.0
2010-01-08    0.0
Name: increase, dtype: float64

In [252]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [253]:
X_train

,log_return,6. volume,positive
date,,,
2010-01-04,0.000000,38409100.0,0.324613
2010-01-05,0.000323,49749600.0,0.715163
2010-01-06,-0.006156,58182400.0,0.610330
2010-01-07,-0.010389,50559700.0,0.774968
2010-01-08,0.006807,51197400.0,0.277817
...,...,...,...
2020-12-08,0.007994,23284094.0,0.149338
2020-12-09,-0.019682,32440603.0,0.119165
2020-12-10,-0.006062,26467017.0,0.389042


In [254]:
from sklearn.ensemble import RandomForestClassifier

In [255]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [256]:
preds = rfc.predict(X_test)

In [257]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [258]:
matrix = confusion_matrix(y_test, preds)

In [259]:
matrix

array([[56, 91],
       [70, 90]])

In [260]:
score = accuracy_score(y_test, preds)

In [261]:
score

0.4755700325732899

In [262]:
y_test.value_counts()

1.0    160
0.0    147
Name: increase, dtype: int64

In [263]:
y_test.value_counts(normalize=True)

1.0    0.521173
0.0    0.478827
Name: increase, dtype: float64

In [264]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

         0.0       0.44      0.38      0.41       147
         1.0       0.50      0.56      0.53       160

    accuracy                           0.48       307
   macro avg       0.47      0.47      0.47       307
weighted avg       0.47      0.48      0.47       307



In [265]:
corr_matrix = df_merge[["log_return", "positive", "increase"]].corr()
print (corr_matrix)

            log_return  positive  increase
log_return  1.000000    0.124373 -0.056121
positive    0.124373    1.000000 -0.010397
increase   -0.056121   -0.010397  1.000000


In [266]:
from statsmodels.tsa.stattools import grangercausalitytests

In [267]:
# #perform Granger-Causality test
# grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])

In [268]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,0.000000,0.401187,0.274200,0.324613
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,0.108230,0.176608,0.715163
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,0.050018,0.339652,0.610330
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,0.029928,0.195104,0.774968
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807,0.319698,0.402485,0.277817


In [269]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966,0.284602,0.617978,0.097420
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935,0.083546,0.596145,0.320308
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610,0.522847,0.434464,0.042689
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326,0.422825,0.495848,0.081327
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0.0,-0.020691,0.965844,0.018870,0.015286


In [270]:
df_merge["return"].mean(), df_merge["return"].std()

(0.0009343558602451958, 0.01585859273755356)

In [271]:
df_merge["log_return"].mean(), df_merge["log_return"].std()

(0.000815047597794545, 0.015857222351919727)

In [272]:
df_merge["increase"] = df_merge["increase"].shift(1)
df_merge["return"] = df_merge["return"].shift(1)

In [273]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,NaN,NaN,0.000000,0.401187,0.274200,0.324613
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,0.000323,1.0,0.000323,0.108230,0.176608,0.715163
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.006137,0.0,-0.006156,0.050018,0.339652,0.610330
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,-0.010335,0.0,-0.010389,0.029928,0.195104,0.774968
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,0.006830,1.0,0.006807,0.319698,0.402485,0.277817


In [274]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,0.004978,1.0,0.004966,0.284602,0.617978,0.097420
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,-0.012852,0.0,-0.012935,0.083546,0.596145,0.320308
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,0.017766,1.0,0.017610,0.522847,0.434464,0.042689
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.014224,0.0,-0.014326,0.422825,0.495848,0.081327
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0.0,-0.020691,0.965844,0.018870,0.015286


In [275]:
df_merge.fillna(0.0, inplace=True)

In [276]:
# df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_BENZINGA_FINBERT_SA.csv")